In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v1.4-alpha2 (30/Nov/2022)


# Carbon Simulation - Demo 4-1

In this demo we investigate the **Alpha Router performance** (a faster albeit possibly less precise routing method than the "exact" router we previously used), and show how it is impacted by setting a threshold to the number of orders that can be routed through.

Initialize an exact simulator and an alpha simulator

In [2]:
Sim = CarbonSimulatorUI(pair="ETHUSDC", verbose=False)
AlphaSim = CarbonSimulatorUI(pair="ETHUSDC", verbose=False, matching_method='alpha')

Add 5 identical strategies to both simulators

In [3]:
strat = ['ETH', 10, 2000, 2500, 1000, 2800, 2700]
for _ in range(5):
    Sim.add_strategy(*strat)
    AlphaSim.add_strategy(*strat)
#Sim.state()['orders']
AlphaSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,1000.0,1000.0,USDC,2800.0,2700.0,2800.0,USDC per ETH,0
2,2,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,3
3,3,ETHUSDC,USDC,1000.0,1000.0,USDC,2800.0,2700.0,2800.0,USDC per ETH,2
4,4,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,5
5,5,ETHUSDC,USDC,1000.0,1000.0,USDC,2800.0,2700.0,2800.0,USDC per ETH,4
6,6,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,7
7,7,ETHUSDC,USDC,1000.0,1000.0,USDC,2800.0,2700.0,2800.0,USDC per ETH,6
8,8,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,2500.0,2000.0,USDC per ETH,9
9,9,ETHUSDC,USDC,1000.0,1000.0,USDC,2800.0,2700.0,2800.0,USDC per ETH,8


In [4]:
# set test amounts
usdc_trade_amount = 1000
eth_trade_amount = 0.5

## Route by Source

#### AMM buys USDC

First we test the standard simulator which should route through all 5 positions

In [5]:
Sim.amm_buys('USDC',usdc_trade_amount, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,0.099895,ETH,200.0,USDC,ETHUSDC,0,1,2002.111456,USDC per ETH
0,0.1,0,1,route #2,False,False,None,0.099895,ETH,200.0,USDC,ETHUSDC,2,1,2002.111456,USDC per ETH
0,0.2,0,2,route #4,False,False,None,0.099895,ETH,200.0,USDC,ETHUSDC,4,1,2002.111456,USDC per ETH
0,0.3,0,3,route #6,False,False,None,0.099895,ETH,200.0,USDC,ETHUSDC,6,1,2002.111456,USDC per ETH
0,0.4,0,4,route #8,False,False,None,0.099895,ETH,200.0,USDC,ETHUSDC,8,1,2002.111456,USDC per ETH
0,0,0,A,AMM sells 0ETH buys 1000USDC,True,False,None,0.499473,ETH,1000.0,USDC,ETHUSDC,"[0, 2, 4, 6, 8]",5,2002.111456,USDC per ETH


Then we can set a threshold on the number of orders using the AlphaSim

In [6]:
AlphaSim.amm_buys('USDC',usdc_trade_amount, execute=False, threshold_orders=3)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,0.0,0,0,route #0,False,False,None,0.166374,ETH,333.333333,USDC,ETHUSDC,0,1,2003.519094,USDC per ETH,3
0,0.1,0,1,route #2,False,False,None,0.166374,ETH,333.333333,USDC,ETHUSDC,2,1,2003.519094,USDC per ETH,3
0,0.2,0,2,route #4,False,False,None,0.166374,ETH,333.333333,USDC,ETHUSDC,4,1,2003.519094,USDC per ETH,3
0,0,0,A,AMM sells 0ETH buys 1000USDC,True,False,None,0.499122,ETH,1000.000000,USDC,ETHUSDC,"[0, 2, 4]",3,2003.519094,USDC per ETH,3


We would expect that routing through fewer orders should give a slightly worse price for the trader.

Since here the amm is buying then the trader is selling 1000 USDC, then the Alpha buy amount of 0.499122 is slightly lower than the 0.49973 ETH achieved in exact routing. However, the difference here is only 7bp

In [7]:
AlphaSim.state()["trades"].query("uid=='0'").iloc[0]["amt1"],Sim.state()["trades"].query("uid=='0'").iloc[0]["amt1"]

(0.499122, 0.499473)

In [8]:
10000*(AlphaSim.state()["trades"].query("uid=='0'").iloc[0]["amt1"]
       /Sim.state()["trades"].query("uid=='0'").iloc[0]["amt1"]-1)

-7.027406886858101

#### AMM buys ETH

In [9]:
Sim.amm_buys('ETH',eth_trade_amount, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #1,False,False,None,278.594364,USDC,0.1,ETH,ETHUSDC,1,1,2785.943638,USDC per ETH
0,1.1,1,1,route #3,False,False,None,278.594364,USDC,0.1,ETH,ETHUSDC,3,1,2785.943638,USDC per ETH
0,1.2,1,2,route #5,False,False,None,278.594364,USDC,0.1,ETH,ETHUSDC,5,1,2785.943638,USDC per ETH
0,1.3,1,3,route #7,False,False,None,278.594364,USDC,0.1,ETH,ETHUSDC,7,1,2785.943638,USDC per ETH
0,1.4,1,4,route #9,False,False,None,278.594364,USDC,0.1,ETH,ETHUSDC,9,1,2785.943638,USDC per ETH
0,1,1,A,AMM sells 1393USDC buys 0ETH,True,False,None,1392.971819,USDC,0.5,ETH,ETHUSDC,"[1, 3, 5, 7, 9]",5,2785.943638,USDC per ETH


In [10]:
AlphaSim.amm_buys('ETH',eth_trade_amount, execute=False, threshold_orders=3)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,1.0,1,0,route #1,False,False,None,462.775145,USDC,0.166667,ETH,ETHUSDC,1,1,2776.650873,USDC per ETH,3
0,1.1,1,1,route #3,False,False,None,462.775145,USDC,0.166667,ETH,ETHUSDC,3,1,2776.650873,USDC per ETH,3
0,1.2,1,2,route #5,False,False,None,462.775145,USDC,0.166667,ETH,ETHUSDC,5,1,2776.650873,USDC per ETH,3
0,1,1,A,AMM sells 1388USDC buys 0ETH,True,False,None,1388.325436,USDC,0.500000,ETH,ETHUSDC,"[1, 3, 5]",3,2776.650873,USDC per ETH,3


Again, we expect that routing through fewer orders should give a slightly worse price for the trader.

Since here the amm is buying then the trader is selling 0.5 ETH, then the Alpha price of 1388.32 USDC means that the return on 0.5 ETH sold is 1388.32 USDC. By comparison, the standard route gives a return of 1392.97 USDC which is slightly better for the trader (33bp)

In [11]:
(AlphaSim.state()["trades"].query("uid=='1'").iloc[0]["amt1"],
 Sim.state()["trades"].query("uid=='1'").iloc[0]["amt1"])

(1388.325436, 1392.971819)

In [12]:
10000*(AlphaSim.state()["trades"].query("uid=='1'").iloc[0]["amt1"]
       /Sim.state()["trades"].query("uid=='1'").iloc[0]["amt1"]-1)

-33.35590093513585

## Route by Target

#### AMM sells USDC

First we test the standard simulator which should route through all 5 positions

In [13]:
Sim.amm_sells('USDC',usdc_trade_amount, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #1,False,False,None,200.0,USDC,0.071687,ETH,ETHUSDC,1,1,2789.909083,USDC per ETH
0,2.1,2,1,route #3,False,False,None,200.0,USDC,0.071687,ETH,ETHUSDC,3,1,2789.909083,USDC per ETH
0,2.2,2,2,route #5,False,False,None,200.0,USDC,0.071687,ETH,ETHUSDC,5,1,2789.909083,USDC per ETH
0,2.3,2,3,route #7,False,False,None,200.0,USDC,0.071687,ETH,ETHUSDC,7,1,2789.909083,USDC per ETH
0,2.4,2,4,route #9,False,False,None,200.0,USDC,0.071687,ETH,ETHUSDC,9,1,2789.909083,USDC per ETH
0,2,2,A,AMM sells 1000USDC buys 0ETH,True,False,None,1000.0,USDC,0.358435,ETH,ETHUSDC,"[1, 3, 5, 7, 9]",5,2789.909083,USDC per ETH


Then we can set a threshold on the number of orders using the AlphaSim

In [14]:
AlphaSim.amm_sells('USDC',usdc_trade_amount, execute=False, threshold_orders = 3)['trades'] # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,2.0,2,0,route #1,False,False,None,333.333333,USDC,0.119767,ETH,ETHUSDC,1,1,2783.181806,USDC per ETH,3
0,2.1,2,1,route #3,False,False,None,333.333333,USDC,0.119767,ETH,ETHUSDC,3,1,2783.181806,USDC per ETH,3
0,2.2,2,2,route #5,False,False,None,333.333333,USDC,0.119767,ETH,ETHUSDC,5,1,2783.181806,USDC per ETH,3
0,2,2,A,AMM sells 1000USDC buys 0ETH,True,False,None,1000.000000,USDC,0.359301,ETH,ETHUSDC,"[1, 3, 5]",3,2783.181806,USDC per ETH,3


We would expect that routing through fewer orders should give a slightly worse price for the trader.

Since here the amm is selling then the trader is buying 1000 USDC, then the Alpha price of 0.359301 ETH means the trader is required to pay more ETH for the same 1000 USDC than the standard approach (0.358435 ETH).

This means that the higher ETH amount is a worse price as this is the amount that the trader will part with for a fixed target USDC amount (24bp)

In [15]:
(AlphaSim.state()["trades"].query("uid=='2'").iloc[0]["amt2"],
 Sim.state()["trades"].query("uid=='2'").iloc[0]["amt2"])

(0.359301, 0.358435)

In [16]:
10000*(AlphaSim.state()["trades"].query("uid=='2'").iloc[0]["amt2"]
       /Sim.state()["trades"].query("uid=='2'").iloc[0]["amt2"]-1)

24.160586996246902

#### AMM sells ETH

In [17]:
Sim.amm_sells('ETH',eth_trade_amount, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,False,None,0.1,ETH,200.211369,USDC,ETHUSDC,0,1,2002.113688,USDC per ETH
0,3.1,3,1,route #2,False,False,None,0.1,ETH,200.211369,USDC,ETHUSDC,2,1,2002.113688,USDC per ETH
0,3.2,3,2,route #4,False,False,None,0.1,ETH,200.211369,USDC,ETHUSDC,4,1,2002.113688,USDC per ETH
0,3.3,3,3,route #6,False,False,None,0.1,ETH,200.211369,USDC,ETHUSDC,6,1,2002.113688,USDC per ETH
0,3.4,3,4,route #8,False,False,None,0.1,ETH,200.211369,USDC,ETHUSDC,8,1,2002.113688,USDC per ETH
0,3,3,A,AMM sells 0ETH buys 1001USDC,True,False,None,0.5,ETH,1001.056844,USDC,ETHUSDC,"[0, 2, 4, 6, 8]",5,2002.113688,USDC per ETH


In [18]:
AlphaSim.amm_sells('ETH',eth_trade_amount, execute=False, threshold_orders = 3)['trades'] # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,3.0,3,0,route #0,False,False,None,0.166667,ETH,333.920883,USDC,ETHUSDC,0,1,2003.525297,USDC per ETH,3
0,3.1,3,1,route #2,False,False,None,0.166667,ETH,333.920883,USDC,ETHUSDC,2,1,2003.525297,USDC per ETH,3
0,3.2,3,2,route #4,False,False,None,0.166667,ETH,333.920883,USDC,ETHUSDC,4,1,2003.525297,USDC per ETH,3
0,3,3,A,AMM sells 0ETH buys 1002USDC,True,False,None,0.500000,ETH,1001.762648,USDC,ETHUSDC,"[0, 2, 4]",3,2003.525297,USDC per ETH,3


Again, we expect that routing through fewer orders should give a slightly worse price for the trader.

Since here the amm is selling then the trader is buying 0.5 ETH, then the Alpha price of 1001.762648 USDC means the trader is required to pay more USDC for the same 0.5 ETH than the standard approach (1001.056844 USDC).

This means that the higher USDC amount is a worse price as this is the amount that the trader will part with for a fixed target ETH amount (7bp)

In [19]:
(AlphaSim.state()["trades"].query("uid=='3'").iloc[0]["amt2"],
 Sim.state()["trades"].query("uid=='3'").iloc[0]["amt2"])

(1001.762648, 1001.056844)

In [20]:
10000*(AlphaSim.state()["trades"].query("uid=='3'").iloc[0]["amt2"]
       /Sim.state()["trades"].query("uid=='3'").iloc[0]["amt2"]-1)

7.050588627712795

### Decreasing the threshold number of orders

It follows that if we decrease the threshold then the price should get worse and vice-versa

In [21]:
AlphaSim.amm_sells('ETH',eth_trade_amount, execute=False, threshold_orders = 2)['trades'] # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,4.0,4,0,route #0,False,False,None,0.25,ETH,501.323152,USDC,ETHUSDC,0,1,2005.292609,USDC per ETH,2
0,4.1,4,1,route #2,False,False,None,0.25,ETH,501.323152,USDC,ETHUSDC,2,1,2005.292609,USDC per ETH,2
0,4,4,A,AMM sells 0ETH buys 1003USDC,True,False,None,0.50,ETH,1002.646305,USDC,ETHUSDC,"[0, 2]",2,2005.292609,USDC per ETH,2


In [22]:
AlphaSim.amm_sells('ETH',eth_trade_amount, execute=False, threshold_orders = 4)['trades'] # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit,threshold_orders
0,5.0,5,0,route #0,False,False,None,0.125,ETH,250.330351,USDC,ETHUSDC,0,1,2002.642808,USDC per ETH,4
0,5.1,5,1,route #2,False,False,None,0.125,ETH,250.330351,USDC,ETHUSDC,2,1,2002.642808,USDC per ETH,4
0,5.2,5,2,route #4,False,False,None,0.125,ETH,250.330351,USDC,ETHUSDC,4,1,2002.642808,USDC per ETH,4
0,5.3,5,3,route #6,False,False,None,0.125,ETH,250.330351,USDC,ETHUSDC,6,1,2002.642808,USDC per ETH,4
0,5,5,A,AMM sells 0ETH buys 1001USDC,True,False,None,0.500,ETH,1001.321404,USDC,ETHUSDC,"[0, 2, 4, 6]",4,2002.642808,USDC per ETH,4


And indeed reducing to 2 orders gives a worse price (i.e. costs more) and increasing to 4 orders gives a better price (i.e. costs less; 13bp)

In [23]:
(AlphaSim.state()["trades"].query("uid=='4'").iloc[0]["amt2"],
 AlphaSim.state()["trades"].query("uid=='5'").iloc[0]["amt2"])

(1002.646305, 1001.321404)

In [24]:
10000*(AlphaSim.state()["trades"].query("uid=='4'").iloc[0]["amt2"]
       / AlphaSim.state()["trades"].query("uid=='5'").iloc[0]["amt2"]-1)

13.231525808869993